<a href="https://colab.research.google.com/github/Cristian2820/Senales-y-sistemas/blob/main/Parciales/Parcial%202/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok


up to date, audited 23 packages in 473ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached websoc

In [ ]:
%%writefile Punto2.py
#Código Dashboard
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import joblib
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# Título de la aplicación
st.title('Parcial 2 20204-1')

# Descripción o texto en la app
st.write("Punto 2")

# Ingreso de la amplitud A y la frecuencia Fo
A = float(st.text_input("Ingrese un valor de entero positivo para A:", value=1))
Fo = float(st.text_input("Ingrese un valor de frecuencia positiva para Fo:", value=1))

# Simulación de la señal de entrada
ti = -1 / (2 * Fo)  # tiempo inicial
tf = 1 / (2 * Fo)   # tiempo final
T = tf - ti         # periodo total (duración del intervalo)
wo = 2 * np.pi / T  # frecuencia angular
st.write('Frecuencia angular (wo):', wo)
st.write('Frecuencia (Fo):', Fo)

Fs = 30 * Fo  # frecuencia de muestreo (respetar Nyquist, Fs >= 2Fo)
tv = np.arange(ti, tf, 1 / Fs)  # vector de tiempo
Nm = len(tv)  # número de muestras en el tiempo

# Señal de entrada |A sin(2πF_ot)|^2
xo = abs(A * np.sin(2 * np.pi * Fo * tv))**2  # señal |A sin(2πF_ot)|^2

# Gráfico de la señal
st.write("Gráfica de la señal de entrada:")
st.line_chart(xo)

# Simulación del espectro y la reconstrucción
# Construcción de bases serie exponencial
N = 10  # número de armónicos
wo = 2 * np.pi / T  # frecuencia fundamental

# Definir bases
phin = np.zeros((Nm, 2 * N + 1), dtype=np.complex_)  # matriz para guardar bases
for n in range(-N, N + 1, 1):
    phin[:, n + N] = np.exp(1j * n * wo * tv)  # base de Fourier en el intervalo de interés

# Espectro de la señal - Serie exponencial
# Calcular el espectro para la señal |A sin(2πF_ot)|^2
Cn = np.zeros(2 * N + 1, dtype=np.complex_)
nv = np.linspace(-N, N, 2 * N + 1)  # vector de número de armónicos
Cn[N] = A**2 / 2  # nivel DC C_0
Cn[N - 2] = A**2 / 4  # armónico en -2
Cn[N + 2] = A**2 / 4  # armónico en +2

# Convertir la magnitud del espectro a decibelios
magnitud_dB = 20 * np.log10(abs(Cn) + 1e-12)  # +1e-12 para evitar logaritmo de cero

# Graficar espectro: Magnitud en dB, Fase, Parte Real y Parte Imaginaria
st.write("Espectro de la señal:")

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Magnitud en decibelios
axs[0, 0].stem(nv, magnitud_dB, 'r', use_line_collection=True)
axs[0, 0].set_xlabel(r'$nw_o$[rad/s]')
axs[0, 0].set_ylabel('Magnitud [dB]')
axs[0, 0].set_title('Magnitud del Espectro en Decibelios')
axs[0, 0].grid()

# Fase en radianes
axs[0, 1].stem(nv, np.angle(Cn), 'r', use_line_collection=True)
axs[0, 1].set_xlabel(r'$nw_o$[rad/s]')
axs[0, 1].set_ylabel('Fase [rad]')
axs[0, 1].set_title('Fase del Espectro')
axs[0, 1].grid()

# Parte Real
axs[1, 0].stem(nv, np.real(Cn), 'b', use_line_collection=True)
axs[1, 0].set_xlabel(r'$nw_o$[rad/s]')
axs[1, 0].set_ylabel('Real$\{C_n\}$')
axs[1, 0].set_title('Parte Real del Espectro')
axs[1, 0].grid()

# Parte Imaginaria
axs[1, 1].stem(nv, np.imag(Cn), 'g', use_line_collection=True)
axs[1, 1].set_xlabel(r'$nw_o$[rad/s]')
axs[1, 1].set_ylabel('Imaginario$\{C_n\}$')
axs[1, 1].set_title('Parte Imaginaria del Espectro')
axs[1, 1].grid()

st.pyplot(fig)

# Reconstrucción de la señal usando las bases de Fourier
x_reconstruida = np.zeros(Nm, dtype=np.complex_)  # Definir como complejo
for n in range(-N, N + 1):
    x_reconstruida += Cn[n + N] * np.exp(1j * n * wo * tv)
x_reconstruida = np.real(x_reconstruida)  # Solo la parte real

# Cálculo del error relativo
error_relativo = np.abs(xo - x_reconstruida) / np.abs(xo)

# Graficar error relativo
st.write("Error relativo entre la señal original y la reconstruida:")
st.line_chart(error_relativo)

# Mostrar el error relativo máximo y promedio
st.write(f"Error relativo máximo: {np.max(error_relativo):.4f}")
st.write(f"Error relativo promedio: {np.mean(error_relativo):.4f}")


Overwriting Punto2.py


In [ ]:
token="2loMihbb5sGRhDJt5EHXjiJaVsd_3tHfRgkUbZitpjScYWL4G"

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Punto2.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://a3ab-34-125-231-142.ngrok-free.app


In [ ]:
#!streamlit run Punto2.py &>/content/logs.txt & curl ipv4.icanhazip.com #Password del Dashboard

In [ ]:
#!npx localtunnel --port 8501 #Link del Dashboard